In [1]:
#IMPORT STATEMENTS
import pandas as pd

In [3]:
#read by chunks
#for chunk in pd.read_csv("2011.csv", chunksize=10):
    #print(chunk)

In [4]:
#read large dataset structure
import dask.dataframe as dd

In [5]:
df_shape = dd.read_csv('2011.csv')

In [6]:
df_shape

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,object,object,object,int64,object,object,float64,object,object,object,object,int64,int64,int64,object,object,object,bool,float64,int64,float64,float64,float64,float64,float64,float64,object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [2]:
#filter out by needed columns
df = pd.read_csv('2011.csv', usecols=['date', 'tweet', 'hashtags'])
df

,date,tweet,hashtags
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw']
1,01/01/2011,In your go-kit should be your pets vaccination...,[]
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[]
3,01/01/2011,Have you found ways to detox after a vaccinati...,[]
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1']
...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']"
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']"
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']"
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']"


In [3]:
#check for null values
df.isnull().sum()

date        0
tweet       0
hashtags    0
dtype: int64

In [4]:
#dropping duplicate rows
#analysis:
print(len(df.index)) #117595 total tweets
df_dup = df.duplicated().tolist()
print(df_dup.count(True)) #9512 duplicated user tweets
df_dup = df.duplicated(['tweet']).tolist()
print(df_dup.count(True)) #17999 duplicated full texts

117595
9512
17999


In [5]:
df = df.drop_duplicates(['tweet']) #drop duplicate tweets
df

,date,tweet,hashtags
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw']
1,01/01/2011,In your go-kit should be your pets vaccination...,[]
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[]
3,01/01/2011,Have you found ways to detox after a vaccinati...,[]
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1']
...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']"
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']"
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']"
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']"


In [6]:
# CLEANUP COMPILER
import re

In [7]:
#cleaning out text:

def cleanTweet(tweet):
    tweet.strip()
    tweet = re.sub(r'@\w+: ?', '', tweet) #cleanout mentions:
    tweet = re.sub(r'@\w* ?', '', tweet) #clean out mentions
    tweet = re.sub(r'#\w+ ?', '', tweet) #remove hashtags symbol
    tweet = re.sub(r'https?:\/\/\S+', '', tweet) #remove hyperlinks
    tweet = re.sub(r'\s{2,}|\t', '', tweet) #remove blank spaces pattern
    tweet = re.sub(r'(RT|rt|FAV|fav|VIA|via)', '', tweet) #remove reserved words
    return tweet

# Decision not to deEmojify emoticons as vaderSentiment works well with emojis
def deEmojify(tweet):
     emojis = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags = re.UNICODE)
     return emojis.sub(r'',tweet)

In [8]:
#make new column for cleaned tweets
df['clean_tweets'] = df['tweet'].apply(cleanTweet)

/Users/sharonhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df.head()

,date,tweet,hashtags,clean_tweets
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...
1,01/01/2011,In your go-kit should be your pets vaccination...,[],In your go-kit should be your pets vaccination...
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox after a vaccination?
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1'],AMESSI - La grippe A(H1N1) relance le débat su...


In [10]:
#CHECK CLEANED TWEET DUPLICATES
print(len(df.index)) #99596 total tweets
df_dup = df.duplicated().tolist()
print(df_dup.count(True)) #no duplicated user tweets
df_dup = df.duplicated(['clean_tweets']).tolist()
print(df_dup.count(True)) #17531 duplicated full texts

99596
0
17351


In [11]:
df = df.drop_duplicates(['clean_tweets'])
df #82245 tweets

,date,tweet,hashtags,clean_tweets
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...
1,01/01/2011,In your go-kit should be your pets vaccination...,[],In your go-kit should be your pets vaccination...
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox after a vaccination?
4,01/01/2011,AMESSI - La grippe A(H1N1) relance le débat su...,['#h1n1'],AMESSI - La grippe A(H1N1) relance le débat su...
...,...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']",FDA clears Pfizer vaccine for adults over 50 -...
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']",More parents pass on shots - Massillon Indepen...
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']",Pet health ale: Rabies vaccine mandatory in th...
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']",CDC expands access to public flu vaccine - Tai...


In [12]:
#START LANGUAGE DETECTION, after noticed some tweets in french
from langdetect import detect

In [13]:
df['lang'] = df['tweet'].map(lambda x: detect(x)) # TAKES A LONG TIME

/Users/sharonhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# CLEAN OUT TWEETS BY LANGUAGE - ENGLISH ONLY:
df = df[df['lang'] == 'en']
df #82245 tweets to 74997 tweets

,date,tweet,hashtags,clean_tweets,lang
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...,en
1,01/01/2011,In your go-kit should be your pets vaccination...,[],In your go-kit should be your pets vaccination...,en
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...,en
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox after a vaccination?,en
5,01/01/2011,RT @daffynitions: Vaccination: the medical sac...,[],Vaccination: the medical sacrament correspond...,en
...,...,...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']",FDA clears Pfizer vaccine for adults over 50 -...,en
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']",More parents pass on shots - Massillon Indepen...,en
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']",Pet health ale: Rabies vaccine mandatory in th...,en
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']",CDC expands access to public flu vaccine - Tai...,en


In [15]:
#DELETE UNNEEDED COLUMN
del df['lang']

In [17]:
#STRIP WHITE SPACES
df['clean_tweets'] = df['clean_tweets'].str.strip()

/Users/sharonhe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
#CHECK CLEANED TWEET DUPLICATES
print(len(df.index)) #74900 total tweets
df_dup = df.duplicated().tolist()
print(df_dup.count(True)) #no duplicated user tweets
df_dup = df.duplicated(['clean_tweets']).tolist()
print(df_dup.count(True)) #1607 duplicated full texts

74990
0
1607


In [19]:
df = df.drop_duplicates(['clean_tweets'])
df #73383 tweets

,date,tweet,hashtags,clean_tweets
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...
1,01/01/2011,In your go-kit should be your pets vaccination...,[],In your go-kit should be your pets vaccination...
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox after a vaccination?
5,01/01/2011,RT @daffynitions: Vaccination: the medical sac...,[],Vaccination: the medical sacrament correspondi...
...,...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']",FDA clears Pfizer vaccine for adults over 50 -...
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']",More parents pass on shots - Massillon Indepen...
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']",Pet health ale: Rabies vaccine mandatory in th...
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']",CDC expands access to public flu vaccine - Tai...


In [44]:
#TESTING LANGUAGE FEATURE
# df_copy = df.head(20)
# df_copy

In [45]:
#TESTING LANGUAGE
# df_copy['lang'] = df_copy['clean_tweets'].apply(lambda x: detect(x))
# df_copy

In [23]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [20]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [14]:
# #negative sentiment scores
# def sentiment_analyzer_scores_neg(tweet):
#     score = analyser.polarity_scores(tweet)
#     return score['neg']

In [15]:
# #neutral sentiment scores
# def sentiment_analyzer_scores_neu(tweet):
#     score = analyser.polarity_scores(tweet)
#     return score['neu']

In [16]:
# #positive sentiment scores
# def sentiment_analyzer_scores_pos(tweet):
#     score = analyser.polarity_scores(tweet)
#     return score['pos']

In [17]:
# #compound sentiment scores
# def sentiment_analyzer_scores_compound(tweet):
#     score = analyser.polarity_scores(tweet)
#     return score['compound']

In [21]:
# #apply analyzers to columns
# df['neg'] = df['clean_tweets'].apply(sentiment_analyzer_scores_neg)

In [22]:
# df['neu'] = df['clean_tweets'].apply(sentiment_analyzer_scores_neu)

In [23]:
# df['pos'] = df['clean_tweets'].apply(sentiment_analyzer_scores_pos)

In [29]:
# df['compound'] = df['clean_tweets'].apply(sentiment_analyzer_scores_compound)

In [24]:
"""
FASTER METHOD:
"""
sentiment = df['tweet'].apply(lambda x: analyser.polarity_scores(x))
df = pd.concat([df, sentiment.apply(pd.Series)], axis=1)

In [25]:
#calculate expanding mean
df['mean'] = df['compound'].expanding().mean()

In [26]:
#calculate rolling mean
df['rolling_24h'] = df['compound'].rolling(window=24, min_periods=1).mean()

In [27]:
df

,date,tweet,hashtags,clean_tweets,neg,neu,pos,compound,mean,rolling_24h
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...,0.000,1.000,0.000,0.0000,0.000000,0.000000
1,01/01/2011,In your go-kit should be your pets vaccination...,[],In your go-kit should be your pets vaccination...,0.000,1.000,0.000,0.0000,0.000000,0.000000
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...,0.352,0.556,0.093,-0.6369,-0.212300,-0.212300
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox after a vaccination?,0.000,1.000,0.000,0.0000,-0.159225,-0.159225
5,01/01/2011,RT @daffynitions: Vaccination: the medical sac...,[],Vaccination: the medical sacrament correspondi...,0.000,1.000,0.000,0.0000,-0.127380,-0.127380
...,...,...,...,...,...,...,...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']",FDA clears Pfizer vaccine for adults over 50 -...,0.000,0.915,0.085,0.0772,-0.010285,0.101967
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']",More parents pass on shots - Massillon Indepen...,0.000,1.000,0.000,0.0000,-0.010285,0.101967
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']",Pet health ale: Rabies vaccine mandatory in th...,0.000,0.844,0.156,0.3612,-0.010280,0.089079
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']",CDC expands access to public flu vaccine - Tai...,0.163,0.750,0.087,-0.2960,-0.010284,0.056167


In [28]:
# PREPARE FOR MANUAL TAGGING
# AUTOMATED TAGGING:
def findSentiment(compound):
    if compound <= -0.2:
        sentiment = 'Negative'
    elif compound >= 0.2:
        sentiment = 'Positive'
    else:
        sentiment = 'NaN'
    return sentiment

In [29]:
df['sentiment tagging'] = df['compound'].apply(findSentiment)

In [30]:
df

,date,tweet,hashtags,clean_tweets,neg,neu,pos,compound,mean,rolling_24h,sentiment tagging
0,01/01/2011,Check out @CDCFlu pledge page for National Inf...,['#cdcnivw'],Check out pledge page for National Influenza V...,0.000,1.000,0.000,0.0000,0.000000,0.000000,NaN
1,01/01/2011,In your go-kit should be your pets vaccination...,[],In your go-kit should be your pets vaccination...,0.000,1.000,0.000,0.0000,0.000000,0.000000,NaN
2,01/01/2011,Vaccination are legal poison. Try to avoid the...,[],Vaccination are legal poison. Try to avoid the...,0.352,0.556,0.093,-0.6369,-0.212300,-0.212300,Negative
3,01/01/2011,Have you found ways to detox after a vaccinati...,[],Have you found ways to detox after a vaccination?,0.000,1.000,0.000,0.0000,-0.159225,-0.159225,NaN
5,01/01/2011,RT @daffynitions: Vaccination: the medical sac...,[],Vaccination: the medical sacrament correspondi...,0.000,1.000,0.000,0.0000,-0.127380,-0.127380,NaN
...,...,...,...,...,...,...,...,...,...,...,...
117590,31/12/2011,FDA clears Pfizer vaccine for adults over 50 -...,"['#vaccination', '#sf']",FDA clears Pfizer vaccine for adults over 50 -...,0.000,0.915,0.085,0.0772,-0.010285,0.101967,NaN
117591,31/12/2011,More parents pass on shots - Massillon Indepen...,"['#vaccination', '#sf']",More parents pass on shots - Massillon Indepen...,0.000,1.000,0.000,0.0000,-0.010285,0.101967,NaN
117592,31/12/2011,Pet health alert: Rabies vaccine mandatory in ...,"['#vaccination', '#sf']",Pet health ale: Rabies vaccine mandatory in th...,0.000,0.844,0.156,0.3612,-0.010280,0.089079,Positive
117593,31/12/2011,CDC expands access to public flu vaccine - Tai...,"['#vaccination', '#sf']",CDC expands access to public flu vaccine - Tai...,0.163,0.750,0.087,-0.2960,-0.010284,0.056167,Negative


In [31]:
# EXPORT TO EXCEL:
df.to_excel('cleaned-2011tweets.xlsx', index=False, header=True)